In [55]:
# clear variable cache
%reset -f

In [56]:
# Data
import pandas as pd
import numpy as np
import datetime

# Notebook Settings
import os

In [57]:
# set wd
os.getcwd()
os.chdir('/home/ad-frazier/data_science/MSBA_320/final_project/clean')

# set pandas options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## Import GDP Data -- Dataset Core

In [58]:
df_gdp = pd.read_csv('gdp_state_annual_clean.csv')

In [59]:
df_gdp.head(2)

,state,year,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus
0,AK,1997,-62900000.00,2464700000.00,12347800000.00,25810800000.00,11061300000.00
1,AK,1998,-54700000.00,2003900000.00,12889600000.00,24227500000.00,9388700000.00


In [60]:
# Get unique states for join integrity confirmation
print('unique states:', df_gdp.state.unique().shape[0])
print('earliest year:', df_gdp.year.min())
print('earliest year:', df_gdp.year.max())

unique states: 51
earliest year: 1997
earliest year: 2020


In [61]:
df_gdp.shape

(1224, 7)

## Import Personal Income + Population

In [62]:
per_income = pd.read_csv('per_capita_personal_income-population_state_annual_clean.csv')

In [63]:
per_income.head(2)

,year,state,per_capita_personal_income,population
0,1929,AL,319.00,2644000.00
1,1929,AR,303.00,1852000.00


# Join 1: GDP + Personal Income and Population

In [64]:
df_core = df_gdp.merge(per_income, how='inner', on=['state','year'])

df_core.head(2)

,state,year,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,population
0,AL,1997,-274500000.00,6754800000.00,61083800000.00,104811900000.00,37247900000.00,21516.00,4367935.00
1,AL,1998,-292900000.00,6968300000.00,64168600000.00,110212000000.00,39368100000.00,22668.00,4404701.00


In [65]:
mov_col = df_core.pop('population')

df_core.insert(2, 'population', mov_col)

df_core.head(3)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income
0,AL,1997,4367935.00,-274500000.00,6754800000.00,61083800000.00,104811900000.00,37247900000.00,21516.00
1,AL,1998,4404701.00,-292900000.00,6968300000.00,64168600000.00,110212000000.00,39368100000.00,22668.00
2,AL,1999,4430141.00,-388400000.00,7329600000.00,67225100000.00,115680100000.00,41513700000.00,23333.00


In [66]:
df_core['per_cap_gdp_current_dollar'] = df_core['gdp_current_dollar']/df_core['population']

df_core.head(2)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar
0,AL,1997,4367935.00,-274500000.00,6754800000.00,61083800000.00,104811900000.00,37247900000.00,21516.00,23995.76
1,AL,1998,4404701.00,-292900000.00,6968300000.00,64168600000.00,110212000000.00,39368100000.00,22668.00,25021.45


In [67]:
# convert remaing gdp measures to per capita
df_core[['subsidies','taxes_on_production_and_imports_(topi)','gdp_compensation','gdp_gross_operating_surplus']] = df_core[['subsidies','taxes_on_production_and_imports_(topi)','gdp_compensation','gdp_gross_operating_surplus']].div(df_core['population'], axis=0)

df_core.head(2)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar
0,AL,1997,4367935.00,-62.84,1546.45,13984.59,104811900000.00,8527.58,21516.00,23995.76
1,AL,1998,4404701.00,-66.50,1582.01,14568.21,110212000000.00,8937.75,22668.00,25021.45


## Import Poverty Rate

In [68]:
poverty_rate = pd.read_csv('poverty_rate_state_annual_clean.csv')

In [69]:
poverty_rate.head(2)

,state,poverty_rate,year
0,AL,0.15,2020
1,AK,0.13,2020


## JOIN 2: df_core + poverty rate

In [70]:
def magic_merge(df1, df2):
    df1 = df1.merge(df2, how='left', on=['state','year'])
    return df1

In [71]:
df_core = magic_merge(df_core, poverty_rate)

df_core.head(2)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate
0,AL,1997,4367935.00,-62.84,1546.45,13984.59,104811900000.00,8527.58,21516.00,23995.76,0.16
1,AL,1998,4404701.00,-66.50,1582.01,14568.21,110212000000.00,8937.75,22668.00,25021.45,0.14


## Import gini_index

In [72]:
gini_index = pd.read_csv('gini_index_state_annual_clean.csv')

gini_index.head(2)

,state,year,gini_index
0,AL,2006,0.47
1,AL,2007,0.47


In [73]:
df_core.head(2)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate
0,AL,1997,4367935.00,-62.84,1546.45,13984.59,104811900000.00,8527.58,21516.00,23995.76,0.16
1,AL,1998,4404701.00,-66.50,1582.01,14568.21,110212000000.00,8937.75,22668.00,25021.45,0.14


In [74]:
df_core = magic_merge(df_core, gini_index)

df_core.head(2)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index
0,AL,1997,4367935.00,-62.84,1546.45,13984.59,104811900000.00,8527.58,21516.00,23995.76,0.16,NaN
1,AL,1998,4404701.00,-66.50,1582.01,14568.21,110212000000.00,8937.75,22668.00,25021.45,0.14,NaN


## Import Median Age

In [75]:
median_age = pd.read_csv('median_age_state_annual_clean.csv')

median_age.head(2)

,state,year,median_age
0,AL,2005,37.40
1,AL,2006,37.20


In [76]:
df_core = magic_merge(df_core, median_age)

df_core.sample(5)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age
144,DC,1997,567736.00,-852.86,3320.91,63940.63,52503600000.00,26070.22,35739.00,92478.90,0.22,NaN,NaN
481,MI,1998,9847942.00,-80.43,2138.52,19211.77,312688100000.00,10481.76,27430.00,31751.62,0.11,NaN,NaN
926,SC,2011,4662785.00,-122.35,2684.02,20486.32,172075100000.00,13855.95,34479.00,36903.93,0.19,0.46,38.00
195,FL,2000,16047515.00,-78.47,2852.74,17311.68,490429800000.00,10475.15,29466.00,30561.11,0.11,NaN,NaN
102,CO,2003,4528732.00,-191.09,2629.46,24024.43,196126500000.00,16844.34,34856.00,43307.16,0.10,NaN,NaN


## Import Median Income

In [77]:
median_income = pd.read_csv('median_income_state_annual_clean.csv')

median_income.head(2)

,state,year,median_income
0,AL,2005,36879.00
1,AL,2006,38783.00


In [78]:
df_core = magic_merge(df_core, median_income)

### Import Mean hours worked

In [79]:
mean_hrs = pd.read_csv('hrs_worked_state_annual_clean.csv')

mean_hrs.head(5)

,state,year,mean_usual_hours
0,AL,2005,39.80
1,AL,2006,39.80
2,AL,2007,39.80
3,AL,2008,39.40
4,AL,2009,38.70


In [80]:
df_core = magic_merge(df_core, mean_hrs)

df_core.sample(15)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours
887,PA,2020,12989625.00,-2264.20,4308.62,33866.14,771897900000.00,23513.63,60720.00,59424.19,0.11,NaN,NaN,NaN,NaN
818,OK,1999,3437147.00,-176.16,1686.43,14080.54,84223900000.00,8913.21,22412.00,24504.01,0.13,NaN,NaN,NaN,NaN
611,NC,2008,9309449.00,-130.80,2693.32,23559.44,418793400000.00,18863.89,37835.00,44985.84,0.14,0.46,37.00,46549.00,39.50
1172,WY,2017,576248.00,-164.34,5762.45,30649.13,36863700000.00,27724.87,57126.00,63971.94,0.13,0.43,37.50,60434.00,40.00
664,NE,2013,1873369.00,-554.67,3623.26,27556.40,107260700000.00,26630.53,46653.00,57255.51,0.10,0.44,36.30,51440.00,39.20
532,MO,2001,5641142.00,-223.68,2145.24,19814.16,190657100000.00,12061.88,28690.00,33797.61,0.10,NaN,NaN,NaN,NaN
180,DE,2009,891730.00,-200.40,3077.50,28256.65,57849700000.00,33739.81,40856.00,64873.56,0.12,0.43,38.20,56860.00,38.20
417,MA,2006,6410084.00,-191.76,3001.22,32711.90,359270000000.00,20526.27,47750.00,56047.63,0.12,0.46,38.30,59963.00,38.30
1085,WA,2002,6052349.00,-151.35,3506.97,22487.70,245978500000.00,14798.52,33212.00,40641.82,0.11,NaN,NaN,NaN,NaN
971,TN,2008,6247411.00,-104.24,3070.63,22397.60,254242000000.00,15331.58,35059.00,40695.58,0.15,0.47,37.70,43614.00,39.10


## Import Median Monthly house cost

In [81]:
monthly_house_cost = pd.read_csv('median_monthly_house_cost_state_annual_clean.csv')

monthly_house_cost.head(5)

,state,year,median_monthly_house_cost
0,AL,2005,624.00
1,AL,2006,685.00
2,AL,2007,713.00
3,AL,2008,742.00
4,AL,2009,751.00


In [82]:
df_core = magic_merge(df_core, monthly_house_cost)

df_core.sample(5)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost
295,IL,2004,12589773.00,-228.07,3225.20,25311.73,563060800000.00,16414.82,36519.00,44723.67,0.12,NaN,NaN,NaN,NaN,NaN
1172,WY,2017,576248.00,-164.34,5762.45,30649.13,36863700000.00,27724.87,57126.00,63971.94,0.13,0.43,37.50,60434.00,40.00,962.00
432,MD,1997,5157328.00,-118.24,1975.13,17944.62,159372100000.00,11100.57,29432.00,30902.07,0.08,NaN,NaN,NaN,NaN,NaN
849,OR,2006,3670883.00,-148.96,1981.62,23714.70,153593100000.00,16293.55,34716.00,41840.91,0.12,0.44,37.60,46230.00,38.40,1121.00
362,KY,1999,4018053.00,-96.37,2072.55,15619.08,115106800000.00,11052.14,23224.00,28647.41,0.12,NaN,NaN,NaN,NaN,NaN


### Import House Values

In [83]:
house_value = pd.read_csv('house_value_quartiles_state_annual_clean.csv')

house_value.head(5)

,state,year,house_low_value_quartile,house_median_value,house_upper_value_quartile
0,AL,2005,59400.00,97500.00,162500.00
1,AL,2006,64200.00,107000.00,174900.00
2,AL,2007,68900.00,115600.00,187700.00
3,AL,2008,68300.00,121500.00,200600.00
4,AL,2009,67400.00,119600.00,197200.00


In [84]:
df_core = magic_merge(df_core, house_value)

df_core.sample(10)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile
258,IA,2015,3136111.00,-543.16,3533.17,27884.89,180299300000.00,26616.47,46104.00,57491.36,0.10,0.44,38.10,54736.00,38.80,834.00,84900.00,136100.00,208800.00
156,DC,2009,592228.00,-1159.01,5935.55,116652.88,100446000000.00,48177.73,60143.00,169606.98,0.18,0.53,35.10,59290.00,39.80,1841.00,306100.00,443700.00,715300.00
804,OH,2009,11528896.00,-144.79,2862.73,23285.52,482281100000.00,15828.92,35755.00,41832.37,0.13,0.45,38.50,45395.00,37.70,975.00,88600.00,134600.00,197100.00
158,DC,2011,617794.00,-1366.80,5910.87,122823.63,109730400000.00,50248.63,67005.00,177616.49,0.20,0.53,33.40,63124.00,40.40,1874.00,NaN,NaN,NaN
645,ND,2018,771407.00,-1701.18,6591.98,37434.58,59092600000.00,34278.27,55511.00,76603.66,0.10,0.44,35.40,63837.00,40.40,861.00,112300.00,198700.00,291600.00
1129,WV,1998,1815609.00,-37.67,1775.66,12451.69,40014600000.00,7849.54,20447.00,22039.22,0.18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,AR,2018,2997271.00,-269.31,3370.37,23687.05,127306500000.00,15686.04,43661.00,42474.14,0.16,0.48,38.10,47062.00,39.60,682.00,75800.00,133100.00,212600.00
527,MN,2020,5707165.00,-2877.31,4630.25,38284.54,373739300000.00,25448.50,61464.00,65485.98,0.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
826,OK,2007,3634349.00,-125.77,2507.27,20237.10,145752100000.00,17485.44,35711.00,40104.05,0.13,0.46,36.10,41567.00,40.20,710.00,65400.00,103000.00,160700.00
849,OR,2006,3670883.00,-148.96,1981.62,23714.70,153593100000.00,16293.55,34716.00,41840.91,0.12,0.44,37.60,46230.00,38.40,1121.00,161800.00,236600.00,365800.00


### Rent Quartiles

In [85]:
rent_quarts = pd.read_csv('rent_quartiles_state_annual_clean.csv')

rent_quarts.head(5)

,state,year,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile
0,AL,2005,272.00,394.00,541.00
1,AL,2006,282.00,410.00,562.00
2,AL,2007,291.00,427.00,591.00
3,AL,2008,314.00,447.00,619.00
4,AL,2009,322.00,468.00,637.00


In [86]:
df_core = magic_merge(df_core, rent_quarts)

df_core.sample(10)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile
814,OH,2019,11788547.00,-172.18,3938.67,31739.19,693199300000.00,23297.10,49809.00,58802.78,0.12,0.47,39.60,58642.00,38.30,929.00,95800.00,157200.00,244800.00,510.00,662.00,862.00
816,OK,1997,3372917.00,-92.65,1647.30,12936.87,79044400000.00,8943.48,21072.00,23435.03,0.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
748,NV,2001,2098399.00,-144.97,3110.47,21483.90,80429200000.00,13879.44,32461.00,38328.84,0.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
444,MD,2009,5730388.00,-182.95,2950.60,29506.80,299102700000.00,19921.44,48448.00,52195.89,0.10,0.45,37.70,69272.00,38.80,1689.00,218300.00,318600.00,458600.00,687.00,941.00,1260.00
1097,WA,2014,7068026.00,-182.03,5001.30,32154.65,440648000000.00,25369.94,50890.00,62343.86,0.12,0.45,37.50,61366.00,38.30,1293.00,171000.00,266200.00,400000.00,673.00,894.00,1244.00
175,DE,2004,830803.00,-159.48,3149.72,27334.64,51353700000.00,31487.25,38974.00,61812.13,0.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
321,IN,2006,6332669.00,-166.25,2935.35,22246.77,259602800000.00,15978.35,33123.00,40994.22,0.11,0.43,36.30,45394.00,39.10,874.00,82600.00,120700.00,171800.00,396.00,510.00,640.00
153,DC,2006,570681.00,-1152.31,5840.39,107753.72,88125000000.00,41978.97,55805.00,154420.77,0.18,0.54,35.00,51847.00,40.30,1558.00,299000.00,437700.00,701700.00,541.00,822.00,1192.00
350,KS,2011,2872476.00,-347.78,3386.03,25222.14,136358900000.00,19210.47,43015.00,47470.86,0.14,0.44,36.30,48964.00,38.80,894.00,NaN,NaN,NaN,NaN,NaN,NaN
535,MO,2004,5747741.00,-154.90,2362.49,21429.60,217084200000.00,14131.41,31510.00,37768.61,0.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Import Occupant Ratio

In [87]:
house_occ_ratio = pd.read_csv('household_occupants_ratio_state_annual_clean.csv')

house_occ_ratio.head(5)

,state,year,household_renter_occ_ratio,household_owner_occ_ratio
0,AL,2005,0.28,0.72
1,AL,2006,0.26,0.74
2,AL,2007,0.27,0.73
3,AL,2008,0.27,0.73
4,AL,2009,0.29,0.71


In [88]:
df_core = magic_merge(df_core, house_occ_ratio)

df_core.sample(10)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile,household_renter_occ_ratio,household_owner_occ_ratio
253,IA,2010,3051447.00,-435.89,2993.33,23537.69,142233900000.00,20516.79,38312.00,46611.95,0.10,0.43,38.20,47961.00,38.10,819.00,NaN,NaN,NaN,NaN,NaN,NaN,0.25,0.75
1067,VT,2008,624151.00,-120.80,3734.51,24418.61,25999300000.00,13623.15,41022.00,41655.46,0.09,0.43,41.50,52104.00,37.90,1170.00,150200.00,214700.00,300200.00,509.00,682.00,900.00,0.25,0.75
1014,UT,2003,2360137.00,-109.99,2191.48,18702.77,78945400000.00,12665.20,25792.00,33449.50,0.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,AR,2008,2874554.00,-175.37,2519.52,19056.24,99706800000.00,13285.61,32306.00,34686.01,0.15,0.46,37.30,38815.00,39.60,660.00,61500.00,105700.00,170700.00,320.00,435.00,573.00,0.31,0.69
28,AR,2001,2691571.00,-422.32,1902.49,15008.74,70616700000.00,9747.36,23873.00,26236.24,0.18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1007,TX,2020,29217653.00,-2024.86,5275.49,32718.13,1775587800000.00,24802.31,55399.00,60771.06,0.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
190,DE,2019,981297.00,-206.67,4676.97,35248.15,77042200000.00,38792.13,53539.00,78510.58,0.07,0.45,41.40,70176.00,38.10,1182.00,181000.00,261700.00,368800.00,745.00,959.00,1201.00,0.28,0.72
392,LA,2005,4576628.00,-151.22,2806.78,18456.45,199594700000.00,22499.75,30139.00,43611.74,0.18,NaN,35.40,36729.00,40.50,629.00,62200.00,101700.00,160400.00,297.00,429.00,571.00,0.30,0.70
1124,WI,2017,5836461.00,-135.65,3752.22,31088.63,318364200000.00,19842.26,48741.00,54547.47,0.09,0.45,39.50,59305.00,38.70,1020.00,119400.00,178900.00,272400.00,546.00,699.00,891.00,0.30,0.70
62,AZ,2011,6446537.00,-117.41,3027.04,21657.80,260915700000.00,15906.35,35321.00,40473.78,0.17,0.46,36.20,46709.00,38.30,1056.00,NaN,NaN,NaN,NaN,NaN,NaN,0.37,0.63


### Household Occupied Binary

In [89]:
house_occ_bool = pd.read_csv('household_occupancy_binary_state_annual_clean.csv')

house_occ_bool.head(5)

,state,year,households_occupied_ratio
0,AK,2005,0.85
1,AK,2006,0.83
2,AK,2007,0.84
3,AK,2008,0.84
4,AK,2009,0.83


In [90]:
df_core = magic_merge(df_core, house_occ_bool)

df_core.sample(10)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile,household_renter_occ_ratio,household_owner_occ_ratio,households_occupied_ratio
721,NM,1998,1793484.00,-80.40,2046.18,13524.35,50658700000.00,12755.84,21380.00,28245.97,0.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
945,SD,2006,783033.00,-692.31,3065.13,19652.05,32364500000.00,19307.36,35299.00,41332.23,0.11,0.44,37.30,42791.00,40.10,708.00,63600.00,112600.00,169500.00,276.00,422.00,584.00,0.27,0.73,0.89
1022,UT,2011,2818261.00,-123.27,2531.70,23080.05,125565000000.00,19065.59,34190.00,44554.07,0.11,0.42,29.60,55869.00,36.60,1177.00,NaN,NaN,NaN,NaN,NaN,NaN,0.28,0.72,0.89
1086,WA,2003,6104115.00,-173.49,3691.13,23013.51,255370200000.00,15304.58,34253.00,41835.75,0.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
567,MS,2012,2982342.00,-148.71,2672.46,18099.33,100448400000.00,13057.96,33657.00,33681.05,0.22,0.49,36.20,37095.00,39.00,625.00,56800.00,99800.00,169400.00,340.00,504.00,697.00,0.32,0.68,0.85
944,SD,2005,775493.00,-1139.66,2980.43,18948.85,30997400000.00,19181.60,33788.00,39971.22,0.12,NaN,37.00,40310.00,40.10,658.00,57300.00,101700.00,156700.00,258.00,409.00,558.00,0.28,0.72,0.89
925,SC,2010,4633890.00,-110.34,2574.66,19771.92,165860000000.00,13556.60,32852.00,35792.82,0.17,0.46,37.80,42018.00,38.30,838.00,NaN,NaN,NaN,NaN,NaN,NaN,0.31,0.69,0.82
193,FL,1998,15486559.00,-79.18,2713.41,15534.90,429138400000.00,9541.25,26883.00,27710.38,0.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
444,MD,2009,5730388.00,-182.95,2950.60,29506.80,299102700000.00,19921.44,48448.00,52195.89,0.10,0.45,37.70,69272.00,38.80,1689.00,218300.00,318600.00,458600.00,687.00,941.00,1260.00,0.29,0.71,0.89
154,DC,2007,574404.00,-1089.65,6391.67,112629.79,93785400000.00,45342.48,59824.00,163274.28,0.18,0.54,34.90,54317.00,40.70,1665.00,310000.00,450900.00,721200.00,590.00,871.00,1278.00,0.52,0.48,0.88


### Import Student Funding(revenue)


In [91]:
stud_rev = pd.read_csv('tot_revenue_student_state_annual_clean.csv')

stud_rev.head(5)

,state,year,per_pupil_revenue
0,AL,2018,11415
1,AK,2018,19553
2,AZ,2018,10396
3,AR,2018,11733
4,CA,2018,16014


In [92]:
df_core = magic_merge(df_core, stud_rev)

df_core.sample(10)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile,household_renter_occ_ratio,household_owner_occ_ratio,households_occupied_ratio,per_pupil_revenue
57,AZ,2006,6029141.00,-104.36,2868.10,22326.13,245957000000.00,15704.81,34703.00,40794.70,0.14,0.45,34.60,47265.00,39.90,1062.00,148100.00,236500.00,376100.00,506.00,658.00,879.00,0.31,0.69,0.85,9023.00
503,MI,2020,10067664.00,-2142.23,3580.01,29660.23,515119900000.00,20067.78,52724.00,51165.78,0.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202,FL,2007,18367842.00,-121.68,4180.90,22732.58,782712500000.00,15821.40,39594.00,42613.20,0.12,0.47,40.00,47804.00,39.90,1142.00,148600.00,230400.00,367900.00,601.00,794.00,1036.00,0.29,0.71,0.81,10995.00
197,FL,2002,16689370.00,-94.57,3007.21,18441.33,553010300000.00,11781.51,30810.00,33135.48,0.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7474.00
972,TN,2009,6306019.00,-136.04,2835.16,21118.35,252332800000.00,16197.13,34446.00,40014.60,0.17,0.47,37.70,41725.00,38.50,835.00,83600.00,137300.00,215400.00,383.00,532.00,696.00,0.29,0.71,0.88,8769.00
973,TN,2010,6356320.00,-118.61,2959.57,21434.98,257776000000.00,16278.33,35851.00,40554.28,0.17,0.47,38.00,41461.00,38.30,836.00,NaN,NaN,NaN,NaN,NaN,NaN,0.31,0.69,0.87,9029.00
483,MI,2000,9952450.00,-94.12,2335.24,21529.46,352152400000.00,11612.90,30344.00,35383.49,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9507.00
647,ND,2020,778962.00,-4974.44,6973.64,37268.96,54854200000.00,31151.58,60451.00,70419.61,0.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181,DE,2010,899762.00,-263.07,3277.31,28351.39,58163500000.00,33277.58,40942.00,64643.21,0.12,0.44,38.80,55847.00,38.20,1242.00,NaN,NaN,NaN,NaN,NaN,NaN,0.26,0.74,0.81,13513.00
605,NC,2002,8326201.00,-106.10,2237.68,19568.34,297846400000.00,14072.28,27852.00,35772.18,0.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7021.00


### Import per student spending

In [93]:
stud_spend = pd.read_csv('tot_spend_student_state_annual_clean.csv')

stud_spend.head(5)

,state,year,per_pupil_expenditure
0,AL,2018,10107
1,AK,2018,18393
2,AZ,2018,8773
3,AR,2018,10412
4,CA,2018,13831


In [94]:
df_core = magic_merge(df_core, stud_spend)

df_core.sample(15)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile,household_renter_occ_ratio,household_owner_occ_ratio,households_occupied_ratio,per_pupil_revenue,per_pupil_expenditure
270,ID,2003,1363380.00,-163.27,2091.49,16380.83,40490600000.00,11389.71,26728.00,29698.69,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6952.00,6168.00
924,SC,2009,4589872.00,-118.13,2505.39,19696.71,161813500000.00,13170.50,32010.00,35254.47,0.14,0.46,37.50,42442.00,38.40,812.00,78200.00,137500.00,228500.00,390.00,545.00,729.00,0.29,0.71,0.83,10838.00,9080.00
980,TN,2017,6739437.00,-116.48,3538.89,26971.79,349837700000.00,21514.85,44969.00,51909.04,0.11,0.48,38.60,51340.00,39.20,807.00,99700.00,167500.00,271100.00,476.00,666.00,911.00,0.33,0.67,0.87,10558.00,9599.00
38,AR,2011,2938833.00,-201.88,2680.32,19351.22,105768100000.00,14160.18,34279.00,35989.83,0.19,0.47,37.50,38758.00,39.20,674.00,NaN,NaN,NaN,NaN,NaN,NaN,0.32,0.68,0.85,10939.00,9536.00
1151,WV,2020,1789798.00,-1406.92,3501.96,22878.56,75855000000.00,17408.22,44868.00,42381.88,0.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178,DE,2007,871749.00,-156.24,3274.80,29936.03,56612800000.00,31887.05,41733.00,64941.63,0.09,0.43,37.80,54610.00,38.90,1174.00,171200.00,239700.00,361700.00,578.00,753.00,941.00,0.26,0.74,0.85,13792.00,12153.00
264,ID,1997,1228520.00,-116.16,1681.86,13171.30,29496900000.00,9273.11,21506.00,24010.11,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5404.00,4721.00
225,GA,2006,9155813.00,-136.47,2946.12,23624.30,406015600000.00,17911.17,34791.00,44345.12,0.13,0.46,34.60,46832.00,40.30,1033.00,100200.00,156800.00,240600.00,402.00,586.00,762.00,0.31,0.69,0.87,10874.00,9102.00
320,IN,2005,6278616.00,-224.62,2912.63,21529.59,248170500000.00,15308.71,31566.00,39526.31,0.13,NaN,36.10,43993.00,39.30,822.00,78700.00,114400.00,165200.00,387.00,500.00,624.00,0.25,0.75,0.90,10831.00,8929.00
14,AL,2011,4812138.00,-107.73,2418.32,20550.18,183916600000.00,15358.54,35010.00,38219.31,0.15,0.47,38.10,41415.00,38.80,774.00,NaN,NaN,NaN,NaN,NaN,NaN,0.29,0.71,0.85,9534.00,8577.00


### Import Education Level

In [95]:
edu_level = pd.read_csv('edu_level_state_annual_clean.csv')

edu_level.head(5)

,state,year,edu_associate_degree,edu_bachelor_degree,edu_high_school,edu_master_degree,edu_total
0,AL,2008,210308.00,439587.00,801711.00,170625.00,3078499.00
1,AL,2009,218551.00,447402.00,797598.00,168782.00,3115982.00
2,AL,2010,225382.00,439863.00,817024.00,182474.00,3161521.00
3,AL,2011,236162.00,443488.00,818373.00,190977.00,3193078.00
4,AL,2012,238555.00,472037.00,826740.00,193901.00,3209646.00


In [96]:
df_core = magic_merge(df_core, edu_level)

df_core.sample(10)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile,household_renter_occ_ratio,household_owner_occ_ratio,households_occupied_ratio,per_pupil_revenue,per_pupil_expenditure,edu_associate_degree,edu_bachelor_degree,edu_high_school,edu_master_degree,edu_total
19,AL,2016,4930595.00,-111.20,2634.31,22888.54,207368400000.00,16645.84,39014.00,42057.48,0.16,0.48,39.00,46257.00,38.90,728.00,76900.00,136200.00,221600.00,393.00,559.00,757.00,0.30,0.70,0.83,10590.00,9528.00,271030.00,506680.00,831521.00,226164.00,3300713.00
407,LA,2020,4651203.00,-2209.62,3909.68,27125.76,235437400000.00,21792.79,50810.00,50618.60,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
780,NY,2009,19307066.00,-303.61,4496.96,32050.08,1160081300000.00,23842.42,47027.00,60085.84,0.16,0.50,38.10,54659.00,38.60,1352.00,131400.00,306000.00,516700.00,564.00,877.00,1267.00,0.42,0.58,0.90,20660.00,18167.00,1100827.00,2434198.00,3151131.00,1303712.00,13197392.00
233,GA,2014,10077700.00,-125.93,2949.22,26063.65,490079300000.00,19743.14,39969.00,48630.07,0.17,0.48,36.10,49321.00,38.90,989.00,84400.00,147900.00,245700.00,497.00,693.00,915.00,0.37,0.63,0.86,10761.00,9476.00,462543.00,1199733.00,1552824.00,503513.00,6576408.00
845,OR,2002,3513424.00,-130.78,1698.37,20040.93,120665500000.00,12735.61,28749.00,34344.13,0.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8302.00,7491.00,NaN,NaN,NaN,NaN,NaN
404,LA,2017,4678606.00,-146.30,3994.52,25898.31,241704200000.00,21915.05,44074.00,51661.58,0.20,0.49,36.80,46145.00,39.60,708.00,90800.00,162500.00,249300.00,485.00,678.00,896.00,0.33,0.67,0.84,12772.00,11636.00,185409.00,482439.00,896962.00,178692.00,3120277.00
1042,VA,2007,7751000.00,-118.77,3266.62,29578.44,391699800000.00,17809.11,44248.00,50535.39,0.09,0.46,37.00,59562.00,39.90,1257.00,151700.00,262100.00,459800.00,501.00,753.00,1114.00,0.28,0.72,0.90,11803.00,10664.00,NaN,NaN,NaN,NaN,NaN
612,NC,2009,9449566.00,-146.76,3129.69,22362.23,411155800000.00,18165.40,35933.00,43510.55,0.17,0.46,36.90,43674.00,38.80,940.00,94000.00,155500.00,245600.00,410.00,568.00,748.00,0.31,0.69,0.86,8802.00,8225.00,524739.00,1091506.00,1410601.00,380622.00,6150247.00
6,AL,2003,4503491.00,-103.85,1872.92,17116.72,134152600000.00,10902.79,26673.00,29788.58,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7349.00,6581.00,NaN,NaN,NaN,NaN,NaN
1147,WV,2016,1835998.00,-77.56,3048.53,20362.22,70924400000.00,15296.69,37380.00,38629.89,0.18,0.47,42.30,43385.00,38.50,527.00,65900.00,117900.00,190100.00,354.00,521.00,686.00,0.26,0.74,0.81,12877.00,11745.00,95794.00,162243.00,422721.00,78802.00,1291757.00


In [97]:
# convert edu level to percent of population
df_core[['edu_associate_degree','edu_bachelor_degree','edu_high_school','edu_master_degree']] = df_core[['edu_associate_degree','edu_bachelor_degree','edu_high_school','edu_master_degree']].div(df_core['edu_total'], axis=0)

df_core.sample(5)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile,household_renter_occ_ratio,household_owner_occ_ratio,households_occupied_ratio,per_pupil_revenue,per_pupil_expenditure,edu_associate_degree,edu_bachelor_degree,edu_high_school,edu_master_degree,edu_total
360,KY,1997,3952747.00,-59.02,2009.64,14001.73,104546200000.00,10496.65,21388.00,26449.00,0.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5875.00,5213.00,NaN,NaN,NaN,NaN,NaN
431,MA,2020,7022220.00,-2877.35,4623.62,49225.34,582476800000.00,31976.07,77021.00,82947.67,0.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,CT,2003,3484336.00,-133.17,3745.71,28623.53,178702200000.00,19051.23,44997.00,51287.30,0.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12815.00,11436.00,NaN,NaN,NaN,NaN,NaN
686,NH,2011,1321886.00,-149.48,3505.29,27929.26,66153200000.00,18759.48,49131.00,50044.56,0.08,0.43,41.50,62647.00,38.30,1513.00,NaN,NaN,NaN,NaN,NaN,NaN,0.25,0.75,0.84,14928.00,13774.00,0.09,0.21,0.25,0.09,914207.00
1121,WI,2014,5778371.00,-138.74,3589.75,28552.37,295696000000.00,19169.50,44709.00,51172.90,0.11,0.44,39.20,52622.00,38.30,1047.00,109000.00,164700.00,244900.00,512.00,657.00,828.00,0.31,0.69,0.87,12850.00,11538.00,0.10,0.19,0.28,0.07,3898162.00


### Merge PCE Values

In [98]:
pce = pd.read_csv('pce_function_state_annual_clean.csv')

pce.head(5)

,year,state,pce_clothing_footwear_and_related_services,pce_communication,pce_education,pce_financial_services_and_insurance,pce_food_and_beverages_purchased_for_off-premises_consumption,pce_food_services_and_accommodations,pce_furnishings_household_equipment_and_routine_household_maintenance,pce_health,pce_housing_utilities_and_fuels,pce_other_goods_and_services,pce_total,pce_recreation,pce_transportation
0,1997,AK,552000000.00,193700000.00,255200000.00,848300000.00,1450000000.00,1091000000.00,701300000.00,1965500000.00,2457500000.00,1146800000.00,14205600000.00,1451200000.00,1711600000.00
1,1997,AL,3457300000.00,1875300000.00,1156500000.00,5026600000.00,7747300000.00,4278700000.00,3993800000.00,14574600000.00,13361000000.00,4375700000.00,77070900000.00,6744600000.00,9454500000.00
2,1997,AR,2186700000.00,881100000.00,715600000.00,2953100000.00,3952400000.00,2370000000.00,2245300000.00,7505800000.00,7595600000.00,2464000000.00,41941200000.00,3290300000.00,5312800000.00
3,1997,AZ,3830400000.00,1930400000.00,1600000000.00,6674700000.00,8135900000.00,6703500000.00,5342900000.00,13544900000.00,20212800000.00,6564900000.00,96365600000.00,8586500000.00,12454400000.00
4,1997,CA,33257500000.00,15812800000.00,12700500000.00,51494500000.00,54740400000.00,42657600000.00,32307800000.00,106739900000.00,132581500000.00,42047000000.00,676652200000.00,69924000000.00,75773900000.00


In [99]:
# divide PCE values to get per capita PCE
pce.iloc[:, 2:] = pce.iloc[:, 2:].div(df_core['population'], axis=0)

pce.sample(10)

,year,state,pce_clothing_footwear_and_related_services,pce_communication,pce_education,pce_financial_services_and_insurance,pce_food_and_beverages_purchased_for_off-premises_consumption,pce_food_services_and_accommodations,pce_furnishings_household_equipment_and_routine_household_maintenance,pce_health,pce_housing_utilities_and_fuels,pce_other_goods_and_services,pce_total,pce_recreation,pce_transportation
592,2008,NJ,12857.64,7846.29,10488.19,29394.36,25006.96,17216.00,15292.74,60610.20,67313.99,22519.30,342906.57,31085.91,29213.34
585,2008,MO,5739.42,4957.64,3968.96,14079.79,14698.45,11961.37,8196.46,40153.27,33631.65,12258.95,197056.34,18057.04,23420.48
546,2007,OK,590.53,370.23,315.60,1102.82,1126.99,928.23,766.97,3330.61,2847.94,1018.49,16106.43,1337.09,2154.55
881,2014,IL,1280.47,886.96,1070.89,3018.49,2549.44,2384.32,1439.54,7559.17,6710.80,2605.14,38200.79,3198.29,3851.00
1072,2018,AL,8186.32,4342.44,5513.54,19806.77,21229.13,18193.53,11736.75,62260.12,47557.77,17703.82,269991.50,20144.78,27227.95
868,2014,AL,397.80,225.11,249.53,784.58,993.14,742.82,502.94,2779.14,2184.65,711.25,12089.58,911.05,1364.23
885,2014,LA,386.15,185.88,209.60,852.22,966.35,882.21,514.61,2658.85,2018.09,734.55,11800.41,808.58,1412.50
683,2010,MD,5289.51,3683.62,4530.57,11356.59,11422.40,9589.97,6608.75,32684.40,33963.98,11863.54,164288.71,13942.03,14019.39
498,2006,RI,106.76,59.57,85.08,265.81,253.53,224.24,144.35,694.00,628.70,228.52,3468.99,282.63,350.72
733,2011,MA,4386.67,2968.05,3688.86,12441.18,9958.10,8561.22,5108.11,30143.08,23802.36,9071.90,138920.49,12419.83,12113.18


### Import Business Applications data

In [107]:
busi_apps = pd.read_csv('business_apps_state_annual_clean.csv')

busi_apps.head(5)

,sa,geo,year,naics_sector,business_applications,business_applications_from_corporations,high‐propensity_business_applications_ba_wba___business_applications_with_planned_wages
0,A,AK,2005,Total for All NAICS,4770.00,1028.00,2708.00
1,A,AK,2006,Total for All NAICS,5218.00,1142.00,2961.00
2,A,AK,2007,Total for All NAICS,5197.00,1024.00,2855.00
3,A,AK,2008,Total for All NAICS,4902.00,749.00,2428.00
4,A,AK,2009,Total for All NAICS,4452.00,607.00,2122.00


In [108]:
busi_apps = busi_apps.rename(columns={'high‐propensity_business_applications_ba_wba___business_applications_with_planned_wages':'high_propensity_business_application',
                                      'geo':'state'})

In [110]:
df_core = magic_merge(df_core, busi_apps)

df_core.sample(10)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile,household_renter_occ_ratio,household_owner_occ_ratio,households_occupied_ratio,per_pupil_revenue,per_pupil_expenditure,edu_associate_degree,edu_bachelor_degree,edu_high_school,edu_master_degree,edu_total,sa,naics_sector,business_applications,business_applications_from_corporations,high_propensity_business_application
406,LA,2019,4664558.00,-169.51,3893.79,27674.60,254562000000.00,23174.78,47606.00,54573.66,0.18,0.50,37.70,51073.00,39.50,727.00,100600.00,172100.00,265300.00,500.00,714.00,933.00,0.32,0.68,0.83,NaN,NaN,0.06,0.16,0.28,0.06,3140201.00,A,Total for All NAICS,54181.00,2796.00,15197.00
222,GA,2003,8622793.00,-156.31,2520.58,21287.11,342576200000.00,16077.78,30896.00,39729.15,0.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9082.00,7742.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284,ID,2017,1734030.00,-173.18,2766.96,22760.74,71688400000.00,15987.61,41560.00,41342.08,0.12,0.45,36.30,52225.00,38.10,926.00,141200.00,207100.00,305400.00,533.00,698.00,909.00,0.28,0.72,0.87,9066.00,7846.00,0.10,0.18,0.22,0.06,1111788.00,A,Total for All NAICS,17951.00,1532.00,6633.00
647,ND,2020,778962.00,-4974.44,6973.64,37268.96,54854200000.00,31151.58,60451.00,70419.61,0.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,Total for All NAICS,6562.00,482.00,2179.00
400,LA,2013,4628228.00,-165.42,3484.25,24411.59,230832500000.00,22144.50,41376.00,49874.92,0.21,0.49,36.00,44164.00,40.00,680.00,73800.00,140300.00,219700.00,426.00,620.00,839.00,0.33,0.67,0.87,12275.00,10853.00,0.05,0.15,0.28,0.05,3034973.00,A,Total for All NAICS,39235.00,2840.00,13779.00
1016,UT,2005,2457719.00,-112.79,2466.72,20760.63,95355100000.00,15683.65,28927.00,38798.21,0.09,NaN,28.50,47934.00,37.60,1024.00,127000.00,167200.00,233800.00,470.00,585.00,732.00,0.26,0.74,0.91,6769.00,5464.00,NaN,NaN,NaN,NaN,NaN,A,Total for All NAICS,29287.00,9953.00,16160.00
6,AL,2003,4503491.00,-103.85,1872.92,17116.72,134152600000.00,10902.79,26673.00,29788.58,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7349.00,6581.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
735,NM,2012,2090254.00,-133.96,2888.26,21116.72,87925200000.00,18193.34,35749.00,42064.36,0.20,0.47,36.80,42558.00,38.40,760.00,89600.00,157500.00,244000.00,466.00,633.00,845.00,0.31,0.69,0.84,10925.00,9164.00,0.08,0.15,0.22,0.08,1360330.00,A,Total for All NAICS,12421.00,1368.00,5137.00
496,MI,2013,9947308.00,-120.79,3061.35,24322.30,437274400000.00,16696.20,39566.00,43959.07,0.13,0.46,39.60,48273.00,37.70,869.00,65900.00,117500.00,191600.00,484.00,629.00,809.00,0.27,0.73,0.85,12192.00,10649.00,0.09,0.16,0.26,0.08,6645295.00,A,Total for All NAICS,73037.00,11292.00,28845.00
374,KY,2011,4374212.00,-157.99,2861.11,21612.17,171586900000.00,14911.65,34799.00,39226.93,0.16,0.47,38.20,41141.00,38.30,745.00,NaN,NaN,NaN,NaN,NaN,NaN,0.29,0.71,0.87,10391.00,9327.00,0.07,0.12,0.27,0.06,2922350.00,A,Total for All NAICS,25041.00,3093.00,10553.00


In [150]:
df_core.shape

(1176, 37)

In [111]:
df_core.to_csv('../core_df.csv', index=False)

### correlation explore

In [155]:
corr = df_core.corr(method='pearson').abs()
s = corr.unstack()
df_corr = pd.DataFrame(s.sort_values(kind="quicksort")).reset_index()

df_corr.to_csv('first_pass_corr.csv', index=False)

,level_0,level_1,0
364,year,gdp_current_dollar,0.20
365,gdp_current_dollar,year,0.20
366,median_age,poverty_rate,0.20
367,poverty_rate,median_age,0.20
368,gdp_gross_operating_surplus,gdp_current_dollar,0.20
...,...,...,...
747,household_renter_occ_ratio,rent_low_value_quartile,0.50
748,household_owner_occ_ratio,rent_low_value_quartile,0.50
749,rent_low_value_quartile,household_owner_occ_ratio,0.50
750,edu_bachelor_degree,per_pupil_expenditure,0.50
